In [1]:
import pandas as pd 
import requests as req
import datetime as dt
import mysql.connector

In [2]:
API_KEY = 'c577f4969f86ebf533d142953293152b'
CITIES =  ['London','Mumbai','New York','Manila','Paris','Tokyo']

In [8]:
#MYSQL coneection configuration
db_config = {
    'host':'localhost',
    'user':'root',
    'password':'Query@1727',
    'database':'WeatherDB'
}

In [18]:
weather_list=[]

for city in CITIES:
    URL = f'https://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}'
    data = req.get(URL).json()
    
    #To convert Kelvin to Celsuis
    def kelvin_to_celsuis():
        data['main']['temp'] = round(data['main']['temp'] - 273.15,2)
    
    kelvin_to_celsuis()   

    def sunrise_sunset():
        data['sys']['sunrise'] = pd.to_datetime(data['sys']['sunrise'],unit='s')
        data['sys']['sunset'] = pd.to_datetime(data['sys']['sunset'],unit='s')

    sunrise_sunset()

    weather_db = {
    'Name':data['name'],
    'Temperature':data['main']['temp'],
    "Description":data['weather'][0]['description'],
    'Humidity':data['main']['humidity'],
    'AirPressure':data['main']['pressure'],
    'Wind Speed':data['wind']['speed'],
    'Sunrise':data['sys']['sunrise'],
    'Sunset':data['sys']['sunset']
    }

    weather_list.append(weather_db)
    
    print(data)

{'coord': {'lon': -0.1257, 'lat': 51.5085}, 'weather': [{'id': 502, 'main': 'Rain', 'description': 'heavy intensity rain', 'icon': '10n'}], 'base': 'stations', 'main': {'temp': 13.31, 'feels_like': 286.14, 'temp_min': 285.43, 'temp_max': 287.2, 'pressure': 997, 'humidity': 88, 'sea_level': 997, 'grnd_level': 994}, 'visibility': 10000, 'wind': {'speed': 8.75, 'deg': 210, 'gust': 14.4}, 'rain': {'1h': 4.21}, 'clouds': {'all': 100}, 'dt': 1732389908, 'sys': {'type': 2, 'id': 268730, 'country': 'GB', 'sunrise': Timestamp('2024-11-23 07:32:31'), 'sunset': Timestamp('2024-11-23 16:01:51')}, 'timezone': 0, 'id': 2643743, 'name': 'London', 'cod': 200}
{'coord': {'lon': 72.8479, 'lat': 19.0144}, 'weather': [{'id': 721, 'main': 'Haze', 'description': 'haze', 'icon': '50n'}], 'base': 'stations', 'main': {'temp': 25.99, 'feels_like': 299.14, 'temp_min': 299.14, 'temp_max': 299.14, 'pressure': 1012, 'humidity': 61, 'sea_level': 1012, 'grnd_level': 1011}, 'visibility': 2200, 'wind': {'speed': 2.06, 

In [19]:
df = pd.DataFrame(weather_list)

In [20]:
try:
    conn = mysql.connector.connect(**db_config)
    cursor = conn.cursor()
    
    for _, row in df.iterrows():
        cursor.execute("""
            INSERT INTO Weather (Name, Temperature, Description, Humidity, AirPressure, WindSpeed, Sunrise, Sunset)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """,(
        row['Name'],
        row['Temperature'],
        row['Description'],
        row['Humidity'],
        row['AirPressure'],
        row['Wind Speed'],
        row['Sunrise'],
        row['Sunset'],
        ))
    conn.commit()
    print('Data inserted sucessfully!')

except mysql.connector.Error as err:
    print(f'Error: {err}')
    
finally:
    if conn.is_connected():
        print("Connection Successful!")
        cursor.close()
        conn.close()

Data inserted sucessfully!
Connection Successful!


In [7]:
print(df)

       Name  Temperature      Description  Humidity  Air pressure  Wind Speed  \
0    London        13.23  overcast clouds        89           997        8.75   
1    Mumbai        25.99            smoke        61          1012        0.00   
2  New York         9.49    broken clouds        62          1001       12.35   
3    Manila        26.83       few clouds        80          1011        1.03   
4     Paris         9.10        clear sky        74          1011        7.20   
5     Tokyo         8.16  overcast clouds        61          1022        3.90   

              Sunrise              Sunset  
0 2024-11-23 07:32:31 2024-11-23 16:01:51  
1 2024-11-24 01:21:30 2024-11-24 12:29:33  
2 2024-11-23 11:52:48 2024-11-23 21:32:44  
3 2024-11-23 22:01:42 2024-11-24 09:24:11  
4 2024-11-23 07:12:17 2024-11-23 16:02:17  
5 2024-11-23 21:26:04 2024-11-24 07:30:07  
